In [53]:
import tensorflow as tf
import scipy as sp
import numpy as np
import pandas as pd
import re
import hashlib
from sklearn.preprocessing import StandardScaler

import os
from pathlib import Path
import IPython.display as ipd
import cProfile

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
%matplotlib inline

from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import ModelCheckpoint

from tensorflow.python.keras.utils import to_categorical

from sklearn.preprocessing import OrdinalEncoder

from tensorflow.python.keras.models import Sequential

from tensorflow.python.keras.layers import Input, Dense, LSTM

from tensorflow.python.keras.layers import Embedding, Dropout, Activation
from tensorflow.python.keras.layers import BatchNormalization
import sys
sys.path.insert(0, '../src/data')
import get_train_test
import keras
import keras.backend as K
from itertools import product
import cnn_utils


import math


In [55]:
from SGDRScheduler import SGDRScheduler

In [2]:
#keras gpu options
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
#config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [3]:
raw_train, raw_dev = get_train_test.get_train_test('../data/raw/train/audio/',10,2)

1000 : 8.980920599729721 s
2000 : 15.26527488432811 s
3000 : 22.481125728103162 s
4000 : 28.889359498961642 s
5000 : 35.160310695639545 s
6000 : 41.40774187301897 s
7000 : 47.10674593374026 s
8000 : 52.583841299562096 s
9000 : 58.027167612034965 s
10000 : 63.466431972457 s
11000 : 68.96417556034918 s
12000 : 74.45383093647666 s
13000 : 80.43878189848772 s
14000 : 86.29197070110416 s
15000 : 92.69032548949784 s
16000 : 98.30019388733858 s
17000 : 105.30292947932675 s
18000 : 111.21089001919181 s
19000 : 117.15116853429213 s
20000 : 122.6452468884077 s
21000 : 128.13560790613982 s
22000 : 133.70808017073244 s
23000 : 139.170041165162 s
24000 : 144.67104752496206 s
25000 : 150.17331501400207 s
26000 : 152.01433149893867 s
27000 : 153.50729231367575 s
28000 : 155.02397335300378 s
29000 : 156.53389336114327 s
30000 : 158.0716940075438 s
31000 : 159.61757671185782 s
32000 : 161.17514598986338 s


In [6]:
raw_train[0][0].shape

(99, 161)

In [5]:
x_train = np.array([np.pad(x[0], ((0,99 - x[0].shape[0]),(0,0)), 'constant', constant_values=(0)) for x in raw_train])
x_dev = np.array([np.pad(x[0], ((0,99 - x[0].shape[0]),(0,0)), 'constant', constant_values=(0)) for x in raw_dev])

y_train = np.array([x[1] for x in raw_train])
y_dev = np.array([x[1] for x in raw_dev])

In [7]:
x_train.shape

(29220, 99, 161)

In [8]:
# unk_train = np.where(y_train == 'unknown')[0]
unk_dev = np.where(y_dev == 'unknown')[0]

np.random.seed(42)
# unk_train_downsampled = np.random.choice(unk_train, size=unk_train.shape[0] - int(x_train.shape[0]/27), replace=False)
unk_dev_downsampled = np.random.choice(unk_dev, size=unk_dev.shape[0] - int(x_dev.shape[0]/27), replace=False)

# all_train_index = np.where(y_train)[0]
all_dev_index = np.where(y_dev)[0]

# downsampled_train_index = np.setdiff1d(all_train_index,unk_train_downsampled)
downsampled_dev_index = np.setdiff1d(all_dev_index,unk_dev_downsampled)

# x_train = x_train[downsampled_train_index]
x_dev_down = x_dev[downsampled_dev_index]

# y_train = y_train[downsampled_train_index]
y_dev_down = y_dev[downsampled_dev_index]



In [9]:
x_train = x_train.reshape(-1,99,161,1)
x_dev_down = x_dev_down.reshape(-1,99,161,1)

In [10]:
x_train = np.concatenate([x_train,x_train,x_train],axis = 3)

In [10]:
x_train.shape

(29220, 99, 161, 1)

In [12]:
x_dev_down = np.concatenate([x_dev_down,x_dev_down,x_dev_down],axis = 3)

In [11]:
x_dev_down.shape

(1255, 99, 161, 1)

In [13]:
def z_norm(x_train, x_val, x_ens = None, x_test = None):
    print("FUP")
    mean = np.mean(x_train, axis=(0, 1, 2, 3))
    std = np.std(x_train, axis=(0, 1, 2, 3))
    x_train = (x_train - mean) / (std + 1e-7)
    x_val = (x_val - mean) / (std + 1e-7)
    if x_ens != None:
        x_ens = (x_ens - mean) / (std + 1e-7)
    if x_test != None:
        x_test = (x_test - mean) / (std + 1e-7)
    return x_train, x_val, x_ens, x_test


In [14]:
#x_train_norm, x_val_norm, _, _ = z_norm(x_train,x_dev_down)
#x_train, x_dev_down, _, _ = cnn_utils.preprocess_res(x_train,x_dev_down)
x_train, x_dev_down, _, _ = cnn_utils.preprocess_mob(x_train,x_dev_down)

In [ ]:
labels_to_keep = ['yes', 'no', 'up', 'down', 'left',
                    'right', 'on', 'off', 'stop', 'go', 'silence']

for label in labels_to_keep + ['unknown']:
    print (f"{label} dev count: {len([x for x in np.array([x[1] for x in raw_dev ]) if x == label])}")

In [13]:
class_weight = {0: 37,
                1: 37,
                2: 37,
                3: 37,
                4: 37,
                5: 37,
                6: 37,
                7: 37,
                8: 2,
                9: 37,
                10: 37}

In [14]:
class_weight

{0: 37, 1: 37, 2: 37, 3: 37, 4: 37, 5: 37, 6: 37, 7: 37, 8: 2, 9: 37, 10: 37}

In [12]:
x_resh = x_train.reshape(-1, 99*161)
scaler = StandardScaler()

scaler.fit(x_resh)

x_train_scaled = scaler.transform(x_resh).reshape(-1,99,161,1)
x_dev_scaled = scaler.transform(x_dev_down.reshape(-1,99*161)).reshape(-1,99,161,1)



In [15]:
y_train_cat = to_categorical(OrdinalEncoder().fit_transform(y_train.reshape(-1,1)))
y_dev_cat = to_categorical(OrdinalEncoder().fit_transform(y_dev_down.reshape(-1,1)))

In [ ]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights,name="weights").initialized_value()
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

weights = np.ones(11)

weights[8] = (2/37) * (10/8)

wcc = weighted_categorical_crossentropy(weights)

In [49]:
checkpointer = ModelCheckpoint(filepath='./model-{val_acc:.3f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True)


In [56]:
x_train_scaled.shape[0]

29220

In [ ]:
epoch_size = x_train_scaled.shape[0]
batch_size = 1024
schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=3e-3,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.3)

In [20]:
np.random.seed(42)

model = Sequential()
model.add(LSTM(110, return_sequences=True, input_shape=(99,161)))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=False))
model.add(BatchNormalization())
model.add(Dense(110))
model.add(BatchNormalization())
model.add(Dense(110))
model.add(BatchNormalization())
model.add(Dense(11))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',  optimizer='adam',metrics=['categorical_accuracy'])

checkpointer = ModelCheckpoint(filepath='./model-{val_categorical_accuracy:.3f}.hdf5', monitor='val_categorical_accuracy', verbose=1, save_best_only=True)


np.random.seed(42)

model.fit(x_train_scaled.reshape(-1,99,161),y_train_cat, batch_size = 1024, epochs = 50, shuffle = True,class_weight=class_weight,
                        validation_data=(x_dev_scaled.reshape(-1,99,161),y_dev_cat), callbacks=[checkpointer])

Train on 29220 samples, validate on 1255 samples
Epoch 1/50
28672/29220 [============================>.] - ETA: 1s - loss: 2.1395 - categorical_accuracy: 0.1372
Epoch 00001: val_categorical_accuracy improved from -inf to 0.19681, saving model to ./model-0.197.hdf5
29220/29220 [==============================] - 70s 2ms/step - loss: 2.1304 - categorical_accuracy: 0.1378 - val_loss: 2.2574 - val_categorical_accuracy: 0.1968
Epoch 2/50
28672/29220 [============================>.] - ETA: 1s - loss: 1.5745 - categorical_accuracy: 0.1953
Epoch 00002: val_categorical_accuracy improved from 0.19681 to 0.27092, saving model to ./model-0.271.hdf5
29220/29220 [==============================] - 57s 2ms/step - loss: 1.5713 - categorical_accuracy: 0.1961 - val_loss: 2.0509 - val_categorical_accuracy: 0.2709
Epoch 3/50
28672/29220 [============================>.] - ETA: 1s - loss: 1.2306 - categorical_accuracy: 0.2976
Epoch 00003: val_categorical_accuracy improved from 0.27092 to 0.34422, saving model

KeyboardInterrupt: 

In [45]:
resnet_model = cnn_utils.get_aml_model(x_train_scaled)


In [46]:
batch_size = 1


In [47]:
import math

In [58]:
epoch_size = x_train_scaled.shape[0]
batch_size = 64
schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=3e-3,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.3)

In [ ]:
resnet_model.fit(x_train_scaled,y_train_cat, batch_size = 64, epochs = 50, shuffle = True,class_weight=class_weight,
                        validation_data=(x_dev_scaled,y_dev_cat), callbacks=[schedule, checkpointer])

Train on 29220 samples, validate on 1255 samples
Epoch 1/50
 6336/29220 [=====>........................] - ETA: 6:05 - loss: 23.2644 - acc: 0.5663

In [ ]:
model = load_model('./model-0.98.hdf5', custom_objects={'loss': wcc})

In [ ]:
x_dev_scaled.shape

In [52]:
x_pred = resnet_model.predict(x_dev_scaled)

x_best_pred = np.argmax(x_pred,axis=1)

pd.DataFrame(x_best_pred.reshape(-1,1))[0].value_counts()

8     155
1     134
4     124
5     113
3     113
10    111
7     108
6     105
0     103
2      99
9      90
Name: 0, dtype: int64

In [ ]:
x_pred = model.predict(x_dev_scaled)

x_best_pred = np.argmax(x_pred,axis=1)

pd.DataFrame(x_best_pred.reshape(-1,1))[0].value_counts()

In [ ]:
pd.DataFrame(y_dev.reshape(-1,1))[0].value_counts()